# Diversity and Inclusion Company Score Card

In [1]:
import lseg.data as ld
import plotly.offline as offline
from plotly import graph_objs as go
from IPython.display import display
offline.offline.init_notebook_mode(connected=True)

In [2]:
ld.open_session()

<lseg.data.session.Definition object at 0x1dce5679af0 {name='workspace'}>

In [3]:
symbol = 'MSFT.O'

## Company fundamental information

In [4]:
d1 = ld.get_data(symbol, fields = ['TR.CompanyName', 'TR.TRBCIndustryGroup', 'TR.HeadquartersCountry', 'TR.TRBCIndustryGroupCode', 'TR.HQCountryCode'])
inst = {
    'RIC': symbol,
    'Name': d1.iloc[0][1],
    'IndustryGroup': d1.iloc[0][2],
    'Headquarters': d1.iloc[0][3],
    'IndustryCode': d1.iloc[0][4],
    'CountryCode': d1.iloc[0][5],
}

for key, val in inst.items():
    display(f'{key}: {val}')

C:\Users\gsbal\AppData\Local\Temp\ipykernel_41372\3950385165.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\gsbal\AppData\Local\Temp\ipykernel_41372\3950385165.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\gsbal\AppData\Local\Temp\ipykernel_41372\3950385165.py:6: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\gsbal\AppData\Local\Temp\ipykernel_41372\3950385165.py:7: FutureWarning:

Series.__getit

'RIC: MSFT.O'

'Name: Microsoft Corp'

'IndustryGroup: Software & IT Services'

'Headquarters: United States of America'

'IndustryCode: 572010'

'CountryCode: US'

## Use screener to get country and industry peers

In [6]:
# define the ranking function
rFields = ['TR.CommonName', 'TR.TRDIRScore']
def getRank(screenerQuery):
    iall = ld.get_data(screenerQuery, fields = rFields)
    iall_sorted = iall.sort_values('DIR Score', ascending=False).reset_index(drop=True)
    iRank = iall_sorted.loc[iall_sorted['Company Common Name'] == inst['Name']].index[0]
    sRank = str(iRank + 1) + "/" + str(iall_sorted.index.max() + 1)
    return sRank, iall_sorted

In [7]:
# Get overall rank
overallRank, overallList = getRank('SCREEN(U(IN(Equity(active,public,primary))), TR.TRDIRScore>=0, CURN=USD)')
display(f'Overall Rank: {overallRank}')

# Get country rank
countryRank, countryList = getRank('SCREEN(U(IN(Equity(active,public,primary))), TR.TRDIRScore>=0, IN(TR.HQCountryCode,' + inst['CountryCode'] + '), CURN=USD)')
display(f'Country Rank: {countryRank}')

# Get industry rank
industryRank, industryList = getRank('SCREEN(U(IN(Equity(active,public,primary))), TR.TRDIRScore>=0, IN(TR.TRBCIndustryGroupCode,' + str(inst['IndustryCode']) + '), CURN=USD)')
display(f'Industry Group Rank: {industryRank}')

'Overall Rank: 1528/6916'

'Country Rank: 272/1537'

'Industry Group Rank: 87/508'

## Top and Bottom 5 in this industry

In [9]:
industryList[['Company Common Name', 'DIR Score']].iloc[0: 5]

,Company Common Name,DIR Score
0,Accenture PLC,84.0
1,Capgemini SE,75.75
2,Infosys Ltd,72.5
3,Autodesk Inc,71.0
4,Hitit Bilgisayar Hizmetleri AS,70.75


In [10]:
industryList[['Company Common Name', 'DIR Score']].iloc[-5: industryList.shape[0]]

,Company Common Name,DIR Score
503,KPS AG,37.5
504,Nerdy Inc,37.25
505,Microstrategy Inc,37.0
506,Netflix Inc,34.75
507,SHIFT Inc,34.25


## Comparison with Country and Industry benchmarks

In [11]:
aFields = ['TR.CommonName', 'TR.TRDIRScore', 'TR.TRDIRDiversityScore', 'TR.TRDIRPeopleDevelopmentScore', 'TR.TRDIRInclusionScore', 'TR.TRDIRControvScore']
bestInIndustry = industryList.iloc[0]['Instrument']
bestInCountry = countryList.iloc[0]['Instrument']
dnIdata = ld.get_data([symbol, bestInCountry, bestInIndustry], fields = aFields)
compare = dnIdata.transpose()
compare.columns = ['', 'Top Ranked {}'.format(inst['CountryCode']), inst['IndustryGroup']]
compare

,,Top Ranked US,Software & IT Services
Instrument,MSFT.O,WBA.OQ,ACN.N
Company Common Name,Microsoft Corp,Walgreens Boots Alliance Inc,Accenture PLC
DIR Score,59.75,82.0,84.0
DIR Diversity Score,50,68,75
DIR People Development Score,54,67,73
DIR Inclusion Score,73,93,88
DIR Controversies Score,62,100,100


In [12]:
# Plot it in the bar chart
def generateBar(index):
    cName = compare.columns[index]
    return go.Bar(x=compare.index[2:], y=compare[cName][2:], name=compare[cName]['Company Common Name'])

fig = go.Figure([generateBar(0), generateBar(1), generateBar(2)])
fig.update_layout(title='Best in Class Score Comparison')
fig.show()

In [13]:
dFields = ['TR.AnalyticBoardCulturalDiversity(IncludePartialYear=False)', 'TR.CGAnalyticBoardFemale(IncludePartialYear=False)', 'TR.AnalyticExecutiveMembersGenderDiversity(IncludePartialYear=False)', 'TR.TargetsDiversityOpportunity(IncludePartialYear=False)', 'TR.PolicyDiversityOpportunity(IncludePartialYear=False)', 'TR.WomenEmployees(IncludePartialYear=False)', 'TR.WomenManagers(IncludePartialYear=False)', 'TR.NewWomenEmployees(IncludePartialYear=False)']
ddata = ld.get_data([symbol, bestInCountry, bestInIndustry], fields = dFields)
compare = ddata.transpose()
compare.columns = ['', 'Top Ranked {}'.format(inst['CountryCode']), inst['IndustryGroup']]
display('DIVERSITY PILLAR SCORES')
display(compare)

'DIVERSITY PILLAR SCORES'

,,Top Ranked US,Software & IT Services
Instrument,MSFT.O,WBA.OQ,ACN.N
"Board Cultural Diversity, Percent",33.333333,18.181818,100.0
Female on Board,41.666667,36.363636,45.454545
"Executive Members Gender Diversity, Percent",28.571429,50.0,33.333333
Targets Diversity and Opportunity,True,True,True
Policy Diversity and Opportunity,True,True,True
Women Employees,33.1,71.0,48.0
Women Managers,28.4,63.0,33.0
New Women Employees,35.5,67.50844,48.0


In [14]:
iFields = ['TR.EmployeeSatisfaction(IncludePartialYear=False)', 'TR.PolicyCareerDevelopment', 'TR.AvgTrainingHours', 'TR.InternalPromotion', 'TR.MgtTraining', 'TR.AnalyticTrainingCosts', 'TR.PolicySkillsTraining']
idata = ld.get_data([symbol, bestInCountry, bestInIndustry], fields = iFields)
compare = idata.transpose()
compare.columns = ['', 'Top Ranked {}'.format(inst['CountryCode']), inst['IndustryGroup']]
display('INCLUSION PILLAR SCORES')
display(compare)

c:\Users\gsbal\envs\rdlib\lib\site-packages\lseg\data\_tools\_dataframe.py:192: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

c:\Users\gsbal\envs\rdlib\lib\site-packages\lseg\data\_tools\_dataframe.py:192: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

c:\Users\gsbal\envs\rdlib\lib\site-packages\lseg\data\_tools\_dataframe.py:192: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the fu

'INCLUSION PILLAR SCORES'

,,Top Ranked US,Software & IT Services
Instrument,MSFT.O,WBA.OQ,ACN.N
Employee Satisfaction,<NA>,<NA>,78
Policy Career Development,True,True,True
Average Training Hours,<NA>,<NA>,61
Internal Promotion,True,True,True
Management Training,True,True,True
Training Costs Per Employee,<NA>,<NA>,1421.18863
Policy Skills Training,True,True,True
